# Task 2: Sample Bengali dataset and do preprocessing

In [1]:
# Imports
import re
import string
import json
from datetime import datetime
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import Module
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from nltk.corpus import stopwords

device = 'cpu'

import random

torch.manual_seed(123)
torch.cuda.manual_seed(234)
np.random.seed(345)
random.seed(456)
torch.manual_seed(567)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# Sample Bengali dataset

In [2]:
# get the size and number of HOF, NOT sentences in hindi-dataset
print('Hindi - training data')
hindi_train_df = pd.read_csv('../../data/hindi_hatespeech.tsv', sep='\t')
display(hindi_train_df['task_1'].value_counts())

print()

print('Hindi - test data')
hindi_test_df = pd.read_csv('../../data/hasoc2019_hi_test_gold_2919.tsv', sep='\t')
display(hindi_test_df['task_1'].value_counts())

Hindi - training data


HOF    2469
NOT    2196
Name: task_1, dtype: int64


Hindi - test data


NOT    713
HOF    605
Name: task_1, dtype: int64

In [3]:
# Sample data from the Bengali dataset so that it has the same size and distribution as in Hindi dataset.
# We also save and preprocess the remaining bengali data (excluding sample train and test), this can
# be useful for Task 3.

bengali_df = pd.read_csv('../../data/bengali_hatespeech.csv')

# separate HATE and NOT-HATE observations
bengali_hate_df = bengali_df[bengali_df['hate']==1]
bengali_not_df = bengali_df[bengali_df['hate']==0]

# sampling data for train/test/other datasets
bengali_hate_sample_df, bengali_hate_other_df = train_test_split(bengali_hate_df, train_size=2469+605, random_state=1)
bengali_hate_train_df, bengali_hate_test_df = train_test_split(bengali_hate_sample_df, train_size=2469, random_state=2)

bengali_not_sample_df, bengali_not_other_df = train_test_split(bengali_not_df, train_size=2196+713, random_state=3)
bengali_not_train_df, bengali_not_test_df = train_test_split(bengali_not_sample_df, train_size=2196, random_state=4)

# form train data, test data, other-data
bengali_train_df = pd.concat([bengali_hate_train_df, bengali_not_train_df]).sample(frac=1)
bengali_test_df = pd.concat([bengali_hate_test_df, bengali_not_test_df]).sample(frac=1)
bengali_other_df = pd.concat([bengali_hate_other_df, bengali_not_other_df]).sample(frac=1)

In [4]:
# show stats of sample Bengali datasets
print('Bengali - sample training data')
display(bengali_train_df['hate'].value_counts())

print()

print('Bengali - sample test data')
display(bengali_test_df['hate'].value_counts())

print()

print('Bengali - remaining data')
display(bengali_other_df['hate'].value_counts())

Bengali - sample training data


1    2469
0    2196
Name: hate, dtype: int64


Bengali - sample test data


0    713
1    605
Name: hate, dtype: int64


Bengali - remaining data


0    17091
1     6926
Name: hate, dtype: int64

In [5]:
# save sample Bengali datasets
bengali_train_df.to_csv('save/bengali_hatespeech_sample_train.csv', index=False)
bengali_test_df.to_csv('save/bengali_hatespeech_sample_test.csv', index=False)
bengali_other_df.to_csv('save/bengali_hatespeech_other.csv', index=False)

# Preprocessing

In [6]:
train_sentences = bengali_train_df['sentence']
test_sentences = bengali_test_df['sentence']
other_sentences = bengali_other_df['sentence']

In [7]:
def preprocess_texts(sentences):
    # remove user taggings
    user_tag_pattern = re.compile(r'\@\w*')
    sentences = [re.sub(user_tag_pattern, ' ', sentence) for sentence in sentences]
    # lower case
    sentences = [sentence.lower() for sentence in sentences]
    # remove punctuations
    punctuation = string.punctuation[:2] + string.punctuation[3:]
    translator = str.maketrans(punctuation, ' '*len(punctuation))
    def remove_punc(s):
        s = s.translate(translator)
        return s

    sentences = [remove_punc(sentence) for sentence in sentences]
    
    # remove stopwords BENGALI
    # source: https://www.ranks.nl/stopwords/bengali
    stopwords = ['অবশ্য', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অথবা', 'অথচ', 'অর্থাত', 'অন্য', 'আজ', 'আছে', 'আপনার', 
                 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আরও', 'আর', 'আগে', 'আগেই', 'আই', 
                 'অতএব', 'আগামী', 'অবধি', 'অনুযায়ী', 'আদ্যভাগে', 'এই', 'একই', 'একে', 'একটি', 'এখন', 'এখনও', 'এখানে', 
                 'এখানেই', 'এটি', 'এটা', 'এটাই', 'এতটাই', 'এবং', 'একবার', 'এবার', 'এদের', 'এঁদের', 'এমন', 'এমনকী', 'এল', 
                 'এর', 'এরা', 'এঁরা', 'এস', 'এত', 'এতে', 'এসে', 'একে', 'এ', 'ঐ', ' ই', 'ইহা', 'ইত্যাদি', 'উনি', 'উপর', 
                 'উপরে', 'উচিত', 'ও', 'ওই', 'ওর', 'ওরা', 'ওঁর', 'ওঁরা', 'ওকে', 'ওদের', 'ওঁদের', 'ওখানে', 'কত', 'কবে', 
                 'করতে', 'কয়েক', 'কয়েকটি', 'করবে', 'করলেন', 'করার', 'কারও', 'করা', 'করি', 'করিয়ে', 'করার', 'করাই', 
                 'করলে', 'করলেন', 'করিতে', 'করিয়া', 'করেছিলেন', 'করছে', 'করছেন', 'করেছেন', 'করেছে', 'করেন', 'করবেন', 
                 'করায়', 'করে', 'করেই', 'কাছ', 'কাছে', 'কাজে', 'কারণ', 'কিছু', 'কিছুই', 'কিন্তু', 'কিংবা', 'কি', 'কী', 'কেউ', 
                 'কেউই', 'কাউকে', 'কেন', 'কে', 'কোনও', 'কোনো', 'কোন', 'কখনও', 'ক্ষেত্রে', 'খুব	গুলি', 'গিয়ে', 'গিয়েছে', 
                 'গেছে', 'গেল', 'গেলে', 'গোটা', 'চলে', 'ছাড়া', 'ছাড়াও', 'ছিলেন', 'ছিল', 'জন্য', 'জানা', 'ঠিক', 'তিনি', 
                 'তিনঐ', 'তিনিও', 'তখন', 'তবে', 'তবু', 'তাঁদের', 'তাঁাহারা', 'তাঁরা', 'তাঁর', 'তাঁকে', 'তাই', 'তেমন', 'তাকে', 
                 'তাহা', 'তাহাতে', 'তাহার', 'তাদের', 'তারপর', 'তারা', 'তারৈ', 'তার', 'তাহলে', 'তিনি', 'তা', 'তাও', 'তাতে', 
                 'তো', 'তত', 'তুমি', 'তোমার', 'তথা', 'থাকে', 'থাকা', 'থাকায়', 'থেকে', 'থেকেও', 'থাকবে', 'থাকেন', 'থাকবেন', 
                 'থেকেই', 'দিকে', 'দিতে', 'দিয়ে', 'দিয়েছে', 'দিয়েছেন', 'দিলেন', 'দু', 'দুটি', 'দুটো', 'দেয়', 'দেওয়া', 'দেওয়ার', 
                 'দেখা', 'দেখে', 'দেখতে', 'দ্বারা', 'ধরে', 'ধরা', 'নয়', 'নানা', 'না', 'নাকি', 'নাগাদ', 'নিতে', 'নিজে', 'নিজেই', 
                 'নিজের', 'নিজেদের', 'নিয়ে', 'নেওয়া', 'নেওয়ার', 'নেই', 'নাই', 'পক্ষে', 'পর্যন্ত', 'পাওয়া', 'পারেন', 'পারি', 'পারে', 
                 'পরে', 'পরেই', 'পরেও', 'পর', 'পেয়ে', 'প্রতি', 'প্রভৃতি', 'প্রায়', 'ফের', 'ফলে', 'ফিরে', 'ব্যবহার', 'বলতে', 
                 'বললেন', 'বলেছেন', 'বলল', 'বলা', 'বলেন', 'বলে', 'বহু', 'বসে', 'বার', 'বা', 'বিনা', 'বরং', 'বদলে', 'বাদে', 
                 'বার', 'বিশেষ', 'বিভিন্ন	বিষয়টি', 'ব্যবহার', 'ব্যাপারে', 'ভাবে', 'ভাবেই', 'মধ্যে', 'মধ্যেই', 'মধ্যেও', 'মধ্যভাগে', 
                 'মাধ্যমে', 'মাত্র', 'মতো', 'মতোই', 'মোটেই', 'যখন', 'যদি', 'যদিও', 'যাবে', 'যায়', 'যাকে', 'যাওয়া', 'যাওয়ার', 
                 'যত', 'যতটা', 'যা', 'যার', 'যারা', 'যাঁর', 'যাঁরা', 'যাদের', 'যান', 'যাচ্ছে', 'যেতে', 'যাতে', 'যেন', 'যেমন', 
                 'যেখানে', 'যিনি', 'যে', 'রেখে', 'রাখা', 'রয়েছে', 'রকম', 'শুধু', 'সঙ্গে', 'সঙ্গেও', 'সমস্ত', 'সব', 'সবার', 'সহ', 
                 'সুতরাং', 'সহিত', 'সেই', 'সেটা', 'সেটি', 'সেটাই', 'সেটাও', 'সম্প্রতি', 'সেখান', 'সেখানে', 'সে', 'স্পষ্ট', 'স্বয়ং', 
                 'হইতে', 'হইবে', 'হৈলে', 'হইয়া', 'হচ্ছে', 'হত', 'হতে', 'হতেই', 'হবে', 'হবেন', 'হয়েছিল', 'হয়েছে', 'হয়েছেন', 'হয়ে', 
                 'হয়নি', 'হয়', 'হয়েই', 'হয়তো', 'হল', 'হলে', 'হলেই', 'হলেও', 'হলো', 'হিসাবে', 'হওয়া', 'হওয়ার', 'হওয়ায়', 'হন', 
                 'হোক', 'জন', 'জনকে', 'জনের', 'জানতে', 'জানায়', 'জানিয়ে', 'জানানো', 'জানিয়েছে', 'জন্য', 'জন্যওজে', 'জে', 
                 'বেশ', 'দেন', 'তুলে', 'ছিলেন', 'চান', 'চায়', 'চেয়ে', 'মোট', 'যথেষ্ট', 'টি']

    sentences = [[word for word in sentence.split() if word not in stopwords] for sentence in sentences]
    
    return sentences

train_sentences = preprocess_texts(train_sentences)
train_texts = [' '.join(l) for l in train_sentences]
bengali_train_df['sentence'] = train_texts

test_sentences = preprocess_texts(test_sentences)
test_texts = [' '.join(l) for l in test_sentences]
bengali_test_df['sentence'] = test_texts

other_sentences = preprocess_texts(other_sentences)
other_texts = [' '.join(l) for l in other_sentences]
bengali_other_df['sentence'] = other_texts

In [8]:
print('train:')
display(bengali_train_df.head())
print('test:')
display(bengali_test_df.head())
print('other:')
display(bengali_other_df.head())

train:


,sentence,hate,category
20440,সমকামী হুজুর,0,religion
7678,ছাএলীগ সালা দের নিসিদ্দ হক,1,politics
22116,কাওয়া গদি ছারলে বুজবে জুতা কেমনে খায়,0,politics
7368,কাউয়া কাদের বড় মাগীখোর ভিডিও পিক দেখলে বুঝা লু...,1,politics
27690,অপু ভালো কথা ছোট করবেনা,0,"Meme, TikTok and others"


test:


,sentence,hate,category
17168,মহিলাকে রিমানডে,0,crime
17512,তুর রিপাতকে মন চাইছিল ছেড়ে গেলি সাথে মিত্যে অ...,0,crime
21184,হুমায়ুন আজাদ এতো বড় ক্রাক মাতাল,0,religion
22900,বাংাল দেশের সবচেয়ে সাহসি ইউটুবার👍👍👍,0,politics
7063,কাদের ধনের মাথা চোর কাদের একটা ফাটা কেষ্ট কোই ...,1,politics


other:


,sentence,hate,category
13576,ছবিটা চখে পানি বাল একটা ছবি,0,entertainment
974,পাপন খানকির পোলা মেইন শয়তান ওরে লাথথী মাইরা বি...,1,sports
17401,পরিবেশটা সুন্দর ডিসি কেহ গালি ডিসি সরাসরি ঢেলে...,0,crime
19883,ইয়া আল্লাহ ইফতারের সময় দোয়া কবুলের সময় সময় দোয়...,0,religion
9722,মাদারচোদ ইহুধির দালাল বাবরি মসজিদ ভাংলে মন্ধির...,1,"Meme, TikTok and others"


## Prepare vocab set

In [9]:
train_sentences = [text.split() for text in bengali_train_df['sentence']]

# flattened_words: all words in all sentences
flattened_words = [word for sentence in train_sentences for word in sentence]

# word_counter: count occurences of each word
word_counter = Counter(flattened_words)

# V: vocabulary
V = list(set(flattened_words))
vocab_size = len(V)
print(f'vocab_size: {vocab_size}')

# mappings word->id and id->word
word_to_int = {}
int_to_word = {}
for i, word in enumerate(V):
    word_to_int[word] = i
    int_to_word[i] = word

vocab_size: 15231


In [10]:
# save dicts for transformation word <-> int
with open('save/bengali_word_to_int_dict.json', 'w') as f:
    json.dump(word_to_int, f)
with open('save/bengali_int_to_word_dict.json', 'w') as f:
    json.dump(int_to_word, f)
with open('save/bengali_word_counter.json', 'w') as f:
    json.dump(word_counter, f)

In [11]:
bengali_train_df.to_csv('save/bengali_hatespeech_sample_train_preprocessed.csv', index=False)
bengali_test_df.to_csv('save/bengali_hatespeech_sample_test_preprocessed.csv', index=False)
bengali_other_df.to_csv('save/bengali_hatespeech_other_preprocessed.csv', index=False)